# 读取930interval数据

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Ipynb_importer
import seaborn as sns
import datetime
from scipy.stats import norm
sns.set(style="darkgrid")
import logging
import time
import os

In [ ]:
def func(path):
    path_list = []
    for i in os.listdir(path):
        path_list.append(i)
    return path_list

In [ ]:
path = '/data/share/china_stocks/data/main_intervals/stock_interval/930'
stocks_begin_interval = func(path)
t1 = datetime.datetime(2010,1,1)
t2 = datetime.datetime(2019,7,1)

begin_interval_df = pd.DataFrame()
for s in range(len(stocks_begin_interval)):
    if s%100 == 0:
        print(s)
    temp = pd.read_csv(os.path.join(path, stocks_begin_interval[s]))
    temp["time"] = pd.to_datetime(temp["time"])
    temp = temp[(temp["time"]>=t1)&(temp["time"]<t2)]
    begin_interval_df = begin_interval_df.append(temp)

In [ ]:
begin_interval_df = begin_interval_df[["id", "time", "open_mid", "close_mid","type"]]
begin_interval_df = begin_interval_df.reset_index(drop=True)
begin_interval_df["date_time"] = pd.to_datetime(begin_interval_df["time"].dt.floor("d"))
begin_interval_df["unique_symbol"] = begin_interval_df["id"].apply(lambda x:str(x)[-6:])
begin_interval_df["930open_mid"] = begin_interval_df.apply(lambda x: np.nan if x["type"]== -100 else x["open_mid"], axis=1)
begin_interval_df["930close_mid"] = begin_interval_df.apply(lambda x: np.nan if x["type"]== -100 else x["close_mid"], axis=1)
gb = begin_interval_df.groupby(["unique_symbol"])
begin_interval_df["t+1_930open_mid"] = gb["930open_mid"].transform(lambda x: x.shift(-1))
begin_interval_df["t+1_930close_mid"] = gb["930close_mid"].transform(lambda x: x.shift(-1))

In [ ]:
res = begin_interval_df[["date_time", "930open_mid", "930close_mid", "t+1_930open_mid", "t+1_930close_mid", "unique_symbol", "type"]]

In [ ]:
res.to_csv("930_interval.csv")

In [ ]:
res = pd.read_csv("930_interval.csv")

In [ ]:
res["date_time"] = pd.to_datetime(res["date_time"])

In [ ]:
res["type"]

In [ ]:
q1 = res[(res["unique_symbol"] == 2682)] 

In [ ]:
t1 = datetime.datetime(2016,3,31)
q1[(res["date_time"]>t1)]